In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI


In [ ]:
llm = ChatOpenAI(model = "gpt-4o-mini", api_key = "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")


In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
!pip install weaviate-client

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = [model.encode(doc.page_content) for doc in splits]


In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents = splits, embedding = OpenAIEmbeddings())



In [ ]:
import getpass
import os

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
retriever = vectorstore.as_retriever()

In [5]:
from bs4 import BeautifulSoup
import requests
from sentence_transformers import SentenceTransformer
from faiss import IndexFlatL2
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

# Step 1: Fetch HTML Content
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text
else:
    raise Exception(f"Failed to fetch webpage. Status code: {response.status_code}")

# Step 2: Parse HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Extract specific content (e.g., from the main post body)
content = soup.find(class_="post-content")  # Adjust the class to match the blog structure
if content:
    text = content.get_text()
else:
    raise Exception("Failed to find content with the specified class.")

# Step 3: Split Text into Chunks
chunk_size = 1000
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]



In [ ]:
# Step 2: Generate Embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = [model.encode(chunk) for chunk in chunks]


In [ ]:
# Step 3: Vector Store
index = IndexFlatL2(len(embeddings[0]))
index.add(np.array(embeddings))


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")


In [ ]:




def retrieve(query, k=5):
    query_embedding = model.encode(query)
    distances, indices = index.search(np.array([query_embedding]), k)
    return [chunks[i] for i in indices[0]]

def generate_response(context, question):
    input_text = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = llm_model.generate(inputs['input_ids'], max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [ ]:
# Step 5: Query the RAG system
query = "What is Task Decomposition?"
retrieved_docs = retrieve(query)
context = "\n".join(retrieved_docs)
response = generate_response(context, query)

print(response)


In [7]:
import os
from bs4 import BeautifulSoup
import requests
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Step 1: Scrape the webpage content
def fetch_webpage_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        content = soup.find(class_="post-content")  # Adjust class to match webpage
        if content:
            return content.get_text()
        else:
            raise Exception("Content not found on the page!")
    else:
        raise Exception(f"Failed to fetch webpage. Status code: {response.status_code}")

# Step 2: Split text into chunks
def split_text_into_chunks(text, chunk_size=1000, chunk_overlap=200):
    chunks = []
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

# Step 3: Embed chunks and build FAISS index
def build_faiss_index(chunks, embedder_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    embedder = SentenceTransformer(embedder_model_name)
    embeddings = embedder.encode(chunks)
    dimension = embeddings.shape[1]
    
    # Create FAISS index
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index, embedder

# Step 4: Retrieve relevant chunks
def retrieve_relevant_chunks(query, index, chunks, embedder):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(np.array(query_embedding), k=3)
    relevant_chunks = [chunks[idx] for idx in indices[0]]
    return " ".join(relevant_chunks)

# Step 5: Generate answers using Bloom
def generate_answer(context, question, model_name="bigscience/bloom-560m"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(**inputs, max_length=1024, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 6: Build the RAG pipeline
def rag_pipeline(url, question):
    print("Fetching and processing webpage content...")
    content = fetch_webpage_content(url)
    chunks = split_text_into_chunks(content)
    
    print("Building FAISS index...")
    index, embedder = build_faiss_index(chunks)
    
    print("Retrieving relevant context...")
    context = retrieve_relevant_chunks(question, index, chunks, embedder)
    
    print("Generating answer...")
    answer = generate_answer(context, question)
    return answer

# Step 7: Run the chatbot
if __name__ == "__main__":
    url = "https://lilianweng.github.io/posts/2023-06-23-agent/"  # Example blog URL
    question = "What is task decomposition?"
    
    answer = rag_pipeline(url, question)
    print("\nGenerated Answer:")
    print(answer)


Fetching and processing webpage content...
Building FAISS index...


/home/hyarrava/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Retrieving relevant context...
Generating answer...

Generated Answer:
Context: t needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompt

In [11]:
import os
import numpy as np
import faiss
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

# Step 1: Extract text from a PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Step 2: Split text into chunks
def split_text_into_chunks(text, chunk_size=1000, chunk_overlap=200):
    chunks = []
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

# Step 3: Embed chunks and build FAISS index
def build_faiss_index(chunks, embedder_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    embedder = SentenceTransformer(embedder_model_name)
    embeddings = embedder.encode(chunks)
    dimension = embeddings.shape[1]
    
    # Create FAISS index
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index, embedder

# Step 4: Retrieve relevant chunks
def retrieve_relevant_chunks(query, index, chunks, embedder):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(np.array(query_embedding), k=3)
    relevant_chunks = [chunks[idx] for idx in indices[0]]
    return " ".join(relevant_chunks)

# Step 5: Generate answers using Bloom
def generate_answer(context, question, model_name="bigscience/bloom-560m"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(**inputs, max_length=1024, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 6: Build the RAG pipeline for PDF
def rag_pipeline_from_pdf(pdf_path, question):
    print("Extracting text from PDF...")
    content = extract_text_from_pdf(pdf_path)
    chunks = split_text_into_chunks(content)
    
    print("Building FAISS index...")
    index, embedder = build_faiss_index(chunks)
    
    print("Retrieving relevant context...")
    context = retrieve_relevant_chunks(question, index, chunks, embedder)
    
    print("Generating answer...")
    answer = generate_answer(context, question)
    return answer

# Step 7: Run the chatbot for PDF


In [1]:
if __name__ == "__main__":
    pdf_path = "../Data/PC_US_Elections_clean.pdf"  # Path to your PDF file
    question = "What are effects on India if Donald trump wins?"
    
    answer = rag_pipeline_from_pdf(pdf_path, question)
    print("\nGenerated Answer:")
    print(answer)


NameError: name 'rag_pipeline_from_pdf' is not defined